# Import Packages and Files

In [1]:
import dataloader as dl
import saveloader as sl
import rotation as rot
import RotNet as RN

# Load Dataset

In [2]:
trainset, testset, classes = dl.load_cifar("./datasets")

Files already downloaded and verified
Files already downloaded and verified


In [3]:
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, 8000, 128)

# Train RotNet for Rotation Task

In [6]:
# initialize network
net = RN.RotNet(num_classes=4, num_conv_block=4, add_avg_pool=False)
net.to(device)

rot_classes = ['original', '90 rotation', '180 rotation', '270 rotation']
criterion = nn.CrossEntropyLoss()

NameError: name 'device' is not defined